In [1]:
import json
import pymysql as mc
import sqlalchemy
from sqlalchemy import create_engine
from getpass import getpass
import Credentials as cred
import pandas as pd
from pandas import pivot_table
import numpy as np
from numpy.linalg import svd
import scipy
import scipy.sparse
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
pw = getpass()

········


In [3]:
cnx = create_engine('mysql+pymysql://'+cred.user+':'+pw+'@'+cred.host+':3306/'+cred.db_name,echo=False)

In [4]:
review_df = pd.read_sql_query("SELECT business_id, date, review_id, stars, user_id FROM review",cnx)
review_df.head()

,business_id,date,review_id,stars,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,2011-02-25,x7mDIiDB3jEiPGPHOmDzyw,2,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,2012-11-13,dDl8zu1vWPdKGihJrwQbpw,5,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,2014-10-23,LZp4UX5zK3e-c5ZGSeo3kA,1,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,2011-02-25,Er4NBWCmCD4nM8_p1GRdow,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,2014-09-05,jsDu6QEJHbwP2Blom1PLCA,5,msQe1u7Z_XuqjGoqhB0J5g


In [5]:
view_df = pd.read_sql_query("SELECT * FROM tempView2",cnx)
view_df.head()

,business_id,categories
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho..."
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo..."
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw..."
3,6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai"
4,8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants"


In [6]:
df = review_df.merge(view_df,on='business_id')
df.head()

,business_id,date,review_id,stars,user_id,categories
0,iCQpiavjjPzJ5_3gPD5Ebg,2011-02-25,x7mDIiDB3jEiPGPHOmDzyw,2,msQe1u7Z_XuqjGoqhB0J5g,"Pizza, Restaurants"
1,iCQpiavjjPzJ5_3gPD5Ebg,2011-10-25,VluIpojME0yKOcRKI5L0Iw,3,D1_nrBr4dOrs7M82OaBRwQ,"Pizza, Restaurants"
2,iCQpiavjjPzJ5_3gPD5Ebg,2016-05-22,nsB9JAeghk0zOaSulSm9Yw,5,jEjc8HxeCeI22M4di0Y1aw,"Pizza, Restaurants"
3,iCQpiavjjPzJ5_3gPD5Ebg,2014-12-30,pcn01EMERpCpHEcoaohdEg,5,RTiaaHSN80VRTLSdNK5rfQ,"Pizza, Restaurants"
4,iCQpiavjjPzJ5_3gPD5Ebg,2013-03-24,0mFtAPTmInbXHqDjX9eiOg,5,hBmJs49zYcXdV4wo8F5iiA,"Pizza, Restaurants"


In [7]:
user_review_df = df.groupby(by='user_id').count()

In [8]:
frequent_posters = user_review_df[user_review_df['business_id']>500].index.values

In [9]:
condensed_df = df[df['user_id'].isin(frequent_posters)]

In [10]:
print (f'Original rows was {df.shape[0]}, limiting to just frequent posters was {condensed_df.shape[0]} rows')
print (f'Original distinct users was {df["user_id"].nunique()}, frequent posters make up {condensed_df["user_id"].nunique()}')
print (f'Original number restaurants was {df["business_id"].nunique()}, frequent posters make up {condensed_df["business_id"].nunique()} restaurants')

Original rows was 3654797, limiting to just frequent posters was 20725 rows
Original distinct users was 1031840, frequent posters make up 29
Original number restaurants was 57173, frequent posters make up 12725 restaurants


In [11]:
piv_df = pivot_table(condensed_df,values='stars',index='user_id',
                     columns='business_id',aggfunc=np.average)
piv_df.head()

business_id,--6MefnULPED_I942VcFNA,--9e1ONYQuAa-CB_Rrw7Tw,--SrzpvFLwP_YFwB_Cetow,--U98MNlDym2cLn36BBPgQ,--cZ6Hhc9F7VkKXxHMVZSQ,--ujyvoQlwVoBgMYtADiLA,-01XupAWZEXbdNbxNg5mEg,-050d_XIor1NpCuWkbIVaQ,-0DET7VdEQOJVJ_v6klEug,-0RU0vASJOV383dYA8VZgA,...,zwANZuuXMSgBR5UXu19UBw,zwNC-Ow4eIMan2__bS9-rg,zwmKWhyyV-zejEUbQzBtHw,zy5UZvaG4iOLSbxuophtTQ,zyPGYeXF4XKCqNN1pjFWhg,zyw5DjrRks7a8OhmBsgCQQ,zzmIMvqiBJ_-wVKg_OnGpw,zzwhN7x37nyjP0ZM8oiHmw,zzwicjPC9g246MK2M1ZFBA,zzzaIBwimxVej4tY6qFOUQ
user_id,,,,,,,,,,,,,,,,,,,,,
2e5V6M4GNufEnbGJpVdCjw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62GNFh5FySkA3MbrQmnqvg,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
C2C0GPKvzWWnP57Os9eQ0w,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CxDOIDnH8gp9KXzpBHJYXw,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
DK57YibC5ShBmqQl97CKog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


In [12]:
# subtracts the mean from each row
normalized = piv_df.sub(piv_df.mean(axis=1),axis=0)
normalized2 = normalized.sub(normalized.mean(axis=0),axis=1)
normalized2.head()

business_id,--6MefnULPED_I942VcFNA,--9e1ONYQuAa-CB_Rrw7Tw,--SrzpvFLwP_YFwB_Cetow,--U98MNlDym2cLn36BBPgQ,--cZ6Hhc9F7VkKXxHMVZSQ,--ujyvoQlwVoBgMYtADiLA,-01XupAWZEXbdNbxNg5mEg,-050d_XIor1NpCuWkbIVaQ,-0DET7VdEQOJVJ_v6klEug,-0RU0vASJOV383dYA8VZgA,...,zwANZuuXMSgBR5UXu19UBw,zwNC-Ow4eIMan2__bS9-rg,zwmKWhyyV-zejEUbQzBtHw,zy5UZvaG4iOLSbxuophtTQ,zyPGYeXF4XKCqNN1pjFWhg,zyw5DjrRks7a8OhmBsgCQQ,zzmIMvqiBJ_-wVKg_OnGpw,zzwhN7x37nyjP0ZM8oiHmw,zzwicjPC9g246MK2M1ZFBA,zzzaIBwimxVej4tY6qFOUQ
user_id,,,,,,,,,,,,,,,,,,,,,
2e5V6M4GNufEnbGJpVdCjw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.123827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62GNFh5FySkA3MbrQmnqvg,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
C2C0GPKvzWWnP57Os9eQ0w,NaN,1.011748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CxDOIDnH8gp9KXzpBHJYXw,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0.123827,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
DK57YibC5ShBmqQl97CKog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [13]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

In [84]:
data = Dataset.load_builtin('ml-100k')
algo = SVD()
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9416  0.9332  0.9403  0.9349  0.9295  0.9359  0.0045  
MAE (testset)     0.7419  0.7389  0.7391  0.7375  0.7336  0.7382  0.0027  
Fit time          4.75    5.36    4.75    4.67    4.84    4.87    0.25    
Test time         0.15    0.13    0.18    0.14    0.18    0.16    0.02    


{'fit_time': (4.752604961395264,
  5.357784986495972,
  4.745971918106079,
  4.672910213470459,
  4.839473009109497),
 'test_mae': array([0.74194848, 0.73889998, 0.73912439, 0.73746895, 0.73357717]),
 'test_rmse': array([0.94155726, 0.93323207, 0.94028023, 0.93492208, 0.92950486]),
 'test_time': (0.14854812622070312,
  0.13282990455627441,
  0.1845407485961914,
  0.13807177543640137,
  0.18204069137573242)}